## textdata linear regression example
* https://viblo.asia/p/predict-independent-values-with-text-data-using-linear-regression-aWj5314eZ6m

# 라이브러리 IMPORT.

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge


In [11]:
data = pd.read_csv('final_dataset.csv')

In [12]:
data.head()

,Unnamed: 0,상품명,상품가격,상품종류,리뷰제목,리뷰내용,별점,키워드,쿠팡리뷰감정지수,네이버지식인감정지수
0,0,오르카 더 라이트 핏 새부리형 쿨링 마스크 + 필터 10개입,"15,400원",filterchangeable,리뷰처럼 첫 느낌이 만족하네요...대박나세요|오르카 더 라이트 핏 새부리형 쿨링 마...,요즘 마스크는 생활의 필수품이 되었다.\n이제는 마스크는 방역을 넘어서 패션으로 생...,4.5,마스크 오르카 제품 많이 일반 좀더 요즘 리뉴얼 피부 정말 이전 근데 착용 좋은 있...,0.112693,0.160557
1,1,씨에이치티 애니웨이 편한숨 비말 차단 마스크 대형 KF-AD,"16,400원",filterchangeable,씨에이치티 애니웨이 편한숨 비말 차단 마스크 대형 KF-AD|숨쉬기 편해요|씨에이치...,kf마스크 찾는것 중에서\n저렴하면서\n상품평도 괜찮아서 구매\n여름이 오고\nkf...,4.5,마스크 숨쉬기 저는 힘있는 집에 일회용 저렴 분들 얼굴 덴탈 착용 KF94 여름이,0.029940,0.160557
2,2,엔 AIRGUARD 미세먼지 스포츠 마스크 성인용 KF94,"15,250원",filterchangeable,KF 94 .국내산 호흡이 편하고 귀 아프지않음!|엔 AIRGUARD 미세먼지 스포...,추가리뷰 2020 02 21\n미스크 살균가를 샀어요. UV C 램프가 속도가 빠르...,4.5,필터 마스크 같아 다른 그래서 정도 제품 KF 귀가 착용 있어 저는 구매 일단 호흡...,0.044757,0.160557
3,3,루미카 튜닝 마스크 솔리드 L,"25,200원",filterchangeable,루미카 튜닝 마스크 솔리드 L|마스크가 꼭필요한시기에 딱 좋은 마스크발견했어요|강추...,피부에 뾰루지가 없어지질 않아서 이건 어떨까 하고 구입함\n쓰고 휙 버리는 1회용 ...,4.5,마스크 루미카 쓰고 피부 하고 다른 사용하 착용 후기 색상,0.025210,0.160557
4,4,루미카 튜닝 마스크 솔리드 M,"25,200원",filterchangeable,피부에좋고 말하기숨쉬기 편하고 경제적인 루미카 마스크|실제 사용하기에 편합니다|이가...,루미카 마스크 드디어 쿠팡 로켓에서도 살 수 있네요~\n\n연예인들이 쓰는 유명 마...,4.5,일단 쓰면 너무 얼굴 마스크 같아 ^^ 계속 숨쉬기 베이지 답답 이쁘 느낌 핑크 그...,-0.003021,0.160557


### 데이터 정제

In [13]:
def cleanse_data(data):
    data['키워드'].str.lower()
    data['키워드'].fillna('nan', inplace=True)
    data['키워드'].replace('[^a-zA-Z0-9]', ' ', regex = True)
    data["상품가격"].replace("원", "", inplace=True, regex = True)
    data["상품가격"].replace(",", "", inplace=True, regex = True)
    data['상품명'].fillna('nan', inplace=True)
    data['상품종류'].fillna('nan', inplace=True)

In [14]:
cleanse_data(data)

### 훈련 & 검증 데이터 셋 분리 

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    data.drop('별점',axis=1), data['별점'], test_size=0.4, random_state=42)

## 피쳐 전처리
* data['상품명'] tfidfVectorizer 로 벡터화
* data['키워드'] tfidfVectorizer 로 벡터화
* data['상품종류'] DictVectorizer 로 벡터화
* data['상품가격'] 데이터 형태 변환 1d -> 2d
* data['쿠팡리뷰감정지수'] 데이터 형태 변환 1d -> 2d

In [16]:
def preprocess_features(data):
    # Convert a collection of raw documents to a matrix of TF-IDF features with TfidfVectorizer
    vectorizer = TfidfVectorizer(min_df=2, max_features = 10)
    product_name = vectorizer.fit_transform(data['상품명']) 

    vectorizer = TfidfVectorizer(min_df=2, max_features = 10)
    keyowrds = vectorizer.fit_transform(data['키워드']) 
    
    enc = DictVectorizer()
    category = enc.fit_transform(data[['상품종류']].to_dict('records'))
    
    data["상품가격"] = data["상품가격"].astype(float)
    price = data["상품가격"].to_numpy().reshape((len(data["상품가격"]),1)) 


    data["쿠팡리뷰감정지수"] = data["쿠팡리뷰감정지수"].astype(float)
    coupang_sentinment = data["쿠팡리뷰감정지수"].to_numpy().reshape((len(data["쿠팡리뷰감정지수"]),1))


    data["네이버지식인감정지수"] = data["네이버지식인감정지수"].astype(float)
    naver_sentiment = data["네이버지식인감정지수"].to_numpy().reshape((len(data["네이버지식인감정지수"]),1)) 
    x = hstack([product_name, keyowrds, category, price, coupang_sentinment, naver_sentiment])
    return x


In [ ]:
x_train = preprocess_features(x_train)
x_test = preprocess_features(x_test)

## 학습 및 예측

In [10]:
# Classifier: 
clf = Ridge(alpha=1.0, random_state=241)

# The target value (algorithm has to predict) is 별점
y = data['별점']

# train model on data
clf.fit(x_train, y_train) 
rslt = clf.predict(x_test)

In [11]:
rslt

array([4.4920396 , 4.46263417, 4.40510181, 4.27633891, 4.49021318,
       4.49386603, 4.35049173, 4.25770938, 4.36656426, 4.4317676 ,
       4.34775209, 4.59249293, 4.42263549, 4.53660435, 4.59249293,
       4.58664837, 4.3693039 , 4.17880786, 4.17880786, 4.32400858])

## 성능 검증

In [12]:

from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_test, rslt, squared=False)
rms

0.2572783969835136